In [ ]:
import numpy as np
import rasterio as rs
import hickle as hkl
import os
import hickle as hkl
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd



In [ ]:
x = pd.read_csv("../../src/asia_grid.csv")
x = x[x['country'] == 'Australia']
x.reset_index(inplace = True)
x.to_csv("../../src/australia.csv")

In [ ]:
x.shape

In [ ]:
import numpy as np
import scipy

x = rs.open("5714X1349Y_FINAL.tif").read(1)
plt.figure(figsize=(12,10))
sns.heatmap(x)
print(np.mean(x))

In [ ]:
from sklearn.neighbors import kneighbors_graph
import networkx

nb = kneighbors_graph(x, n_neighbors = 3)
g = networkx.from_scipy_sparse_matrix(nb)
l = networkx.k_core(g)
l = networkx.to_numpy_array(l)



In [ ]:
def Func(a):
    a = np.sum(a > 40) #np.sum((a > 5) & (a < 40))
    return a


%time xmax = scipy.ndimage.generic_filter(x, Func, 3)

In [ ]:
sns.heatmap(xmax)

In [ ]:
xnew = np.copy(x)
xnew[xmax < 1 ] = 0.
plt.figure(figsize=(12,10))
sns.heatmap(x - xnew)

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(xnew)